In [ ]:
def solve_WB(cargo):
    """
    Solve Weight and Balance
    """
    m = Model("WB")

    start_time_WB = time.time()
    a_lat_TOW = 0.5
    b_lat_TOW = 0.5
    a_lat_LW = 0.5
    b_lat_LW = 0.5

    '''Decision variables'''

    f = {}
    
    for j in cargo.uld:
        for t in aircraft.loadlocations:
            f[j.index, t.index] = m.addVar(lb=0, vtype=GRB.BINARY, name=f'f_{j.index}_{t.index}')


    m.update()

    '''Model Sense'''

    m.ModelSense = GRB.MINIMIZE

    m.update()
    # BUG 11: este modelo usa: 0, 5 (solo 2 objetivos, pero omite indices 1-4)

    ''' Objective function 1 --> Maximize the %MAC [Highest Priority] '''

    ZFW_index_obj = m.addVar(vtype=GRB.CONTINUOUS, name="ZFW_index_obj")

    m.addConstr(ZFW_index_obj == aircraft.DOI + aircraft.define_INDEX_PAX() +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C1) * aircraft.delta_index_cargo_C1) +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C2) * aircraft.delta_index_cargo_C2) +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C3) * aircraft.delta_index_cargo_C3) +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C4) * aircraft.delta_index_cargo_C4))
    
    m.addConstr(aircraft.define_INDEX_ZFW_fwd(aircraft.aircraft_type) <= ZFW_index_obj, "ZFW_index_fwd_constraint")
    m.addConstr(ZFW_index_obj <= aircraft.define_INDEX_ZFW_aft(aircraft.aircraft_type), "ZFW_index_aft_constraint")

    MAC_obj = (((aircraft.C * (ZFW_index_obj - aircraft.K)) / aircraft.ZFW) + aircraft.reference_arm - aircraft.lemac) / (aircraft.mac_formula / 100)


    m.setObjectiveN(MAC_obj, index = 0, priority = 2, weight = -1)

    m.update()

    '''Objective function 5 --> Minimize the proximity score of BAX ULDs [Low Priority]'''
    
    obj4 = quicksum(aircraft.define_proximity_score_loadlocation(t) * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations if j.isBAX)
    m.setObjectiveN(obj4, index = 5, priority = 1, weight = 1)

    '''Constraints'''
    
    # O1: Asignación de ULD - Every ULD must be assigned to exactly one position
    for j in cargo.uld:
        m.addConstr(quicksum(f[j.index, t.index] for t in aircraft.loadlocations) == 1, name = f'C_combi_2_{j.index}')

    # O2: Posición Única - Each position can hold at most one ULD, except for the BULK position, where several bulk cargo loading devices could be loaded [Positional constraint]
    for t in aircraft.loadlocations:
        m.addConstr(quicksum(f[j.index, t.index] for j in cargo.uld) <= 1, name = f'C4_{t.index}')

    # O3: Posiciones Prohibidas - ULD type must match position type compatibility
    for j in cargo.uld:
        m.addConstr(quicksum(f[j.index, t.index] for t in aircraft.define_forbidden_positions_for_ULD(j)) == 0, name = f'C5_{j.index}')

    # O4: Posiciones Superpuestas - Overlapping positions cannot both be used
    for j_1 in cargo.uld:
        for j_2 in cargo.uld:
            if j_1 != j_2:
                for t_1 in aircraft.loadlocations:
                        for t_2 in aircraft.define_overlapping_positions(t_1):
                            m.addConstr(f[j_1.index, t_1.index] + f[j_2.index, t_2.index] <= 1, name = f'C6_{j_1.index}_{j_2.index}_{t_1.index}_{t_2.index}')

    # O5: Peso por Posición - Weight at position cannot exceed position limit
    for t in aircraft.loadlocations:
            m.addConstr(quicksum(j.weight * f[j.index, t.index] for j in cargo.uld)
                    <= aircraft.define_max_weight_postion(t), name = f'C7_2_{t.index}')

    # O6a: Peso Compartimento C1 - Total weight in compartment 1 limit
    for t in aircraft.loadlocations_C1:
        m.addConstr(quicksum(j.weight * f[j.index, t.index] for j in cargo.uld)
                    <= aircraft.max_weight_C1, name = f'C_Added_1_{t.index}')

    # O6b: Peso Compartimento C2 - Total weight in compartment 2 limit
    for t in aircraft.loadlocations_C2:
        m.addConstr(quicksum(j.weight * f[j.index, t.index] for j in cargo.uld)
                    <= aircraft.max_weight_C2, name = f'C_Added_2_{t.index}')

    # O6c: Peso Compartimento C3 - Total weight in compartment 3 limit
    for t in aircraft.loadlocations_C3:
        m.addConstr(quicksum(j.weight * f[j.index, t.index] for j in cargo.uld)
                    <= aircraft.max_weight_C3, name = f'C_Added_3_{t.index}')

    # O6d: Peso Compartimento C4 - Total weight in compartment 4 limit
    for t in aircraft.loadlocations_C4:
        m.addConstr(quicksum(j.weight * f[j.index, t.index] for j in cargo.uld)
                    <= aircraft.max_weight_C4, name = f'C_Added_4_{t.index}') 
        
    # O6e: Peso Compartimentos C1+C2 - Combined weight in front compartments
    for t in aircraft.loadlocations_C1_C2:
        m.addConstr(quicksum(j.weight * f[j.index, t.index] for j in cargo.uld)
                    <= aircraft.max_weight_C1_C2, name = f'C_Added_5_{t.index}')
        
    # O6f: Peso Compartimentos C3+C4 - Combined weight in rear compartments
    for t in aircraft.loadlocations_C3_C4:
        m.addConstr(quicksum(j.weight * f[j.index, t.index] for j in cargo.uld)
                    <= aircraft.max_weight_C3_C4, name = f'C_Added_6_{t.index}')
        
    # O7: Peso Total (MPL) - Total cargo weight cannot exceed Maximum Payload Limit
    m.addConstr(quicksum(j.weight * f[j.index, t.index] for t in aircraft.loadlocations for j in cargo.uld)
                <= aircraft.define_MPL(), name = f'C8')

    # O8: Balance Lateral TOW - Lateral balance for takeoff weight (left-right and right-left)
    m.addConstr(quicksum(j.weight * f[j_1.index, t_left.index] for j_1 in cargo.uld for t_left in aircraft.loadlocations_left) -
                quicksum(j.weight * f[j_2.index, t_right.index] for j_2 in cargo.uld for t_right in aircraft.loadlocations_right) <= 
                a_lat_TOW * (quicksum(j.weight * f[j.index, t.index] for t in aircraft.loadlocations for j in cargo.uld) + aircraft.OEW + aircraft.TOF) * b_lat_TOW, name = f'C9_1')
    
    m.addConstr(quicksum(j.weight * f[j_2.index, t_right.index] for j_2 in cargo.uld for t_right in aircraft.loadlocations_right) -
                quicksum(j.weight * f[j_1.index, t_left.index] for j_1 in cargo.uld  for t_left in aircraft.loadlocations_left) <= 
                a_lat_TOW * (quicksum(j.weight * f[j.index, t.index] for t in aircraft.loadlocations for j in cargo.uld) + aircraft.OEW + aircraft.TOF) * b_lat_TOW, name = f'C9_2')

    # O9: Balance Lateral LW - Lateral balance for landing weight (left-right and right-left)
    m.addConstr(quicksum(j.weight * f[j_1.index, t_left.index] for j_1 in cargo.uld for t_left in aircraft.loadlocations_left) - 
                quicksum(j.weight * f[j_2.index, t_right.index] for j_2 in cargo.uld for t_right in aircraft.loadlocations_right) <= 
                a_lat_LW * (quicksum(j.weight * f[j.index, t.index] for t in aircraft.loadlocations for j in cargo.uld) + aircraft.OEW + aircraft.TOF - aircraft.TripF) * b_lat_LW, name = f'C10_1')

    m.addConstr(quicksum(j.weight * f[j_2.index, t_right.index] for j_2 in cargo.uld for t_right in aircraft.loadlocations_right) - 
                quicksum(j.weight * f[j_1.index, t_left.index] for j_1 in cargo.uld for t_left in aircraft.loadlocations_left) <= 
                a_lat_LW * (quicksum(j.weight * f[j.index, t.index] for t in aircraft.loadlocations for j in cargo.uld) + aircraft.OEW + aircraft.TOF - aircraft.TripF) * b_lat_LW, name = f'C10_2')

    # O10: Envelope CG TOW - Longitudinal CG envelope for takeoff (forward and aft limits)
    m.addConstr(aircraft.define_INDEX_TOW_fwd(aircraft.aircraft_type) <= aircraft.DOI + aircraft.fuel_index + aircraft.define_INDEX_PAX() +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C1) * aircraft.delta_index_cargo_C1) +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C2) * aircraft.delta_index_cargo_C2) +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C3) * aircraft.delta_index_cargo_C3) + 
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C4) * aircraft.delta_index_cargo_C4), name = 'C11_1')
    
    m.addConstr(aircraft.DOI + aircraft.fuel_index + aircraft.define_INDEX_PAX() +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C1) * aircraft.delta_index_cargo_C1) +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C2) * aircraft.delta_index_cargo_C2) +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C3) * aircraft.delta_index_cargo_C3) +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C4) * aircraft.delta_index_cargo_C4) <= 
                aircraft.define_INDEX_TOW_aft(aircraft.aircraft_type), name = 'C11_2')
    
    # O11: Envelope CG ZFW - Longitudinal CG envelope for zero fuel weight (forward and aft limits)
    m.addConstr(aircraft.define_INDEX_ZFW_fwd(aircraft.aircraft_type) <= aircraft.DOI + aircraft.define_INDEX_PAX() +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C1) * aircraft.delta_index_cargo_C1) +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C2) * aircraft.delta_index_cargo_C2) +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C3) * aircraft.delta_index_cargo_C3) +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C4) * aircraft.delta_index_cargo_C4), name = 'C12_1')
    
    m.addConstr(aircraft.DOI + aircraft.define_INDEX_PAX() +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C1) * aircraft.delta_index_cargo_C1) +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C2) * aircraft.delta_index_cargo_C2) +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C3) * aircraft.delta_index_cargo_C3) +
                (quicksum(j.weight * f[j.index, t.index] for j in cargo.uld for t in aircraft.loadlocations_C4) * aircraft.delta_index_cargo_C4) <= 
                aircraft.define_INDEX_ZFW_aft(aircraft.aircraft_type), name = 'C12_2')
    
    # O12: COL/CRT Especial - COL and CRT cargo cannot be in same compartment (aircraft-specific)
    uld_with_COL = [j.index for j in cargo.uld if j.COL == 1]
    uld_with_CRT = [j.index for j in cargo.uld if j.CRT == 1]

    if str(aircraft.aircraft_type) in ['772', '77W']:
        for t in aircraft.loadlocations_C1_C2:
            m.addConstr(quicksum(f[j, t.index] for j in uld_with_COL) + quicksum(f[k, t.index] for k in uld_with_CRT) <= 1, name=f'C_special_COL_CRT_C1_C2_{t.index}')
        for t in aircraft.loadlocations_C3_C4:
            m.addConstr(quicksum(f[j, t.index] for j in uld_with_COL) + quicksum(f[k, t.index] for k in uld_with_CRT) <= 1, name=f'C_special_COL_CRT_C3_C4_{t.index}')
    
    if str(aircraft.aircraft_type) in ['789', '781']:
        for t in aircraft.loadlocations_C3_C4:
            m.addConstr(quicksum(f[j, t.index] for j in uld_with_COL) + quicksum(f[k, t.index] for k in uld_with_CRT) == 0, name=f'C_special_COL_CRT_{t.index}')

    m.update()

    '''Optimize the model'''
    WB_env = m.getMultiobjEnv(0)
    bax_env = m.getMultiobjEnv(5)

    WB_env.setParam(GRB.Param.TimeLimit, 60)
    bax_env.setParam(GRB.Param.TimeLimit, 15)

    m.optimize()


    status = m.status
    if status == GRB.Status.INF_OR_UNBD or status == GRB.Status.INFEASIBLE: 
        print('The model is infeasible or unbounded')
        # m.computeIIS()
        # m.write('model.ilp')
        print('Infeasibility report written to model.ilp')
    elif status == GRB.Status.TIME_LIMIT:
        print('Time limit reached')
    elif status == GRB.Status.OPTIMAL:
        print('Solution found')
    elif status == GRB.Status.INTERRUPTED:
        print('Optimization was stopped early')


    '''Results'''
    results_filename = 'Results.txt'
    folder_path = project_setup.setup_project_directory(aircraft.flight_number, aircraft.date, aircraft.departure_airport, aircraft.arrival_airport, baseline = False, optimized_actual = True)
    results_file_path = os.path.join(folder_path, results_filename)

    with open(results_file_path, 'w') as file:
        total_weight = 0
        number_of_uld_solution = 0
        for t in aircraft.loadlocations:
            for j in cargo.uld:
                if f[j.index, t.index].x > 0.9999:
                    number_of_uld_solution += 1
                    print(f'ULD {j.serialnumber} with weight {j.weight:.1f} kg is loaded to position {t.location}')
                    file.write(f'ULD {j.serialnumber} with weight {j.weight:.1f} kg is loaded to position {t.location}\n')


        print('---------------------------------------------------------------------------')
        file.write('---------------------------------------------------------------------------\n')
        print(f'Weight in Compartment 1: {sum(j.weight * f[j.index, t.index].x for j in cargo.uld for t in aircraft.loadlocations_C1):.1f} kg')
        file.write(f'Weight in Compartment 1: {sum(j.weight * f[j.index, t.index].x for j in cargo.uld for t in aircraft.loadlocations_C1):.1f} kg\n')
        print(f'Weight in Compartment 2: {sum(j.weight * f[j.index, t.index].x for j in cargo.uld for t in aircraft.loadlocations_C2):.1f} kg')
        file.write(f'Weight in Compartment 2: {sum(j.weight * f[j.index, t.index].x for j in cargo.uld for t in aircraft.loadlocations_C2):.1f} kg\n')
        print(f'Weight in Compartment 3: {sum(j.weight * f[j.index, t.index].x for j in cargo.uld for t in aircraft.loadlocations_C3):.1f} kg')
        file.write(f'Weight in Compartment 3: {sum(j.weight * f[j.index, t.index].x for j in cargo.uld for t in aircraft.loadlocations_C3):.1f} kg\n')
        print(f'Weight in Compartment 4: {sum(j.weight * f[j.index, t.index].x for j in cargo.uld for t in aircraft.loadlocations_C4):.1f} kg')
        file.write(f'Weight in Compartment 4: {sum(j.weight * f[j.index, t.index].x for j in cargo.uld for t in aircraft.loadlocations_C4):.1f} kg\n')

        #Calculating %MAC
        TOW_index = aircraft.DOI + aircraft.fuel_index + aircraft.define_INDEX_PAX()
        ZFW_index = aircraft.DOI + aircraft.define_INDEX_PAX()

        dict_loadlocations = {aircraft.delta_index_cargo_C1: aircraft.loadlocations_C1, aircraft.delta_index_cargo_C2: aircraft.loadlocations_C2, aircraft.delta_index_cargo_C3: aircraft.loadlocations_C3, aircraft.delta_index_cargo_C4: aircraft.loadlocations_C4}

        for j in cargo.uld:
            for value, compartment in dict_loadlocations.items():
                for t in compartment:
                    if f[j.index, t.index].x > 0.9999:
                        TOW_index += j.weight * f[j.index, t.index].x * float(value) 
                        ZFW_index += j.weight * f[j.index, t.index].x * float(value)

        increment_value = aircraft.define_ff_increment_MAC_ZFW(MAC_obj.getValue())
        fuel_saving_kg = aircraft.TripF * (increment_value / 100)

        number_of_uld_solution = number_of_uld_solution - len([j for j in cargo.uld if j.isBAXorBUPorT])

        print('===========================================================================')
        file.write('===========================================================================\n')
        print(f'%MAC ZFW is {MAC_obj.getValue()}')
        file.write(f'%MAC ZFW is {MAC_obj.getValue()}\n')
        print('---------------------------------------------------------------------------')
        file.write('---------------------------------------------------------------------------\n')
        print(f'The actual %MAC ZFW for this flight was {aircraft.actual_MAC_ZFW}')
        file.write(f'The actual %MAC ZFW for this flight was {aircraft.actual_MAC_ZFW}\n')
        print(f'Resulting in a fuel deviation of {increment_value:.3f}% or {fuel_saving_kg:.3f} kg')
        file.write(f'Resulting in a fuel deviation of {increment_value:.3f}% or {fuel_saving_kg:.3f} kg\n')
        print('---------------------------------------------------------------------------')
        file.write('---------------------------------------------------------------------------\n')
        print(f'{number_of_uld_solution} ULDs are built by the model')
        file.write(f'{number_of_uld_solution} ULDs are built by the model\n')
        print(f'{cargo.total_number_of_build_ULDs} ULDs were actually built')
        file.write(f'{cargo.total_number_of_build_ULDs} ULDs were actually built\n')
        print('---------------------------------------------------------------------------')

        plot.WB(aircraft, ZFW_index, TOW_index, folder_path)

        total_time_WB = time.time() - start_time_WB

        model_info_path = os.path.join(folder_path, 'Model_Information.txt')

        with open(model_info_path, 'w') as file:
            file.write('---------------------------------------------------------------------------\n')
            file.write(f'Total time: {total_time_WB:.3f} seconds\n')
            file.write('---------------------------------------------------------------------------\n')

    return

In [ ]:
solve_WB(cargo)